# Lineas de cache
### LEE ATENTAMENTE ESTE `notebook` EJECUTANDO SUS CELDAS. COMPLETA Y CONTESTA LAS CELDAS QUE CORRESPONDAN

Asegúrate de que estás ejecutando el código de este _notebook_ en la máquina y desde el directorio que quieres

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

!echo "HOSTNAME     " `hostname`; 
!echo "CURRENT DIR  " `pwd`

HOSTNAME      scicomp
CURRENT DIR   /mnt/teaching/parallel-computing/ucaldas.20152.msc/student


## Instrucciones

Inspecciona y entiende el siguiente programa en C. Observa como:
* creamos arrays de ciertos tipos de datos
* recorremos varias veces el array con un paso diferente cada vez


### ES RECOMENDABLE QUE EJECUTES LAS CELDAS DE ESTE NOTEBOOK ANTES EMPEZAR TU TAREA
tratando de entender qué está haciendo el código y cómo interpretar los datos obtenidos

# TAREA: Mide el tiempo de ejecución con los arrays de tipo `int`, `long` y `double`</font>
mide los tiempos y almacénalos en las variables `time_int`, `time_long`, `time_double` y `time_long_double` de forma análoga a como se hace con la variable `time_char`.

**$\rightarrow$ modifica el código y sálvalo ** (`shift` + `return`)

In [43]:
%%writefile files/cache_line_example.c

#include <stdlib.h>
#include "rlxutils.h"

int main(int argc, char **argv) {
    struct timespec tsi, tsf;

    long const SIZE = 10*1024*1024;

    char   *array_char   = malloc(SIZE*sizeof(char));
    int    *array_int    = malloc(SIZE*sizeof(int));
    long   *array_long   = malloc(SIZE*sizeof(long));
    double *array_double = malloc(SIZE*sizeof(double));
    long double *array_long_double = malloc(SIZE*sizeof(long double));

    printf("size of char         %lu\n", sizeof(char));
    printf("size of int          %lu\n", sizeof(int));
    printf("size of long         %lu\n", sizeof(long));
    printf("size of double       %lu\n", sizeof(double));
    printf("size of float        %lu\n", sizeof(float));
    printf("size of long double  %lu\n", sizeof(long double));

    int i,j, g;
    for (i = 0; i < SIZE; i++) {
        array_int[i] = 3;
        array_char[i] = 3;
        array_long[i] = 3;
        array_double[i] = 3;
        array_long_double[i] = 3;
    }

    char *fname = "files/cache_line.data";
    printf ("writing to file %s\n", fname);
    FILE *f = fopen(fname, "w");
    fprintf(f, "step_size elapsed_time_char elapsed_time_int elapsed_time_long elapsed_time_double elapsed_time_long_double\n");

    int k[13] = {1,2,4,8,16,32,64,128,256,512,1024, 2048, 4096};
    int N = 10;
    for (j=0; j<13; j++) {

        int step = k[j];
        double time_char = 0;
        double time_int  = 0;
        double time_long = 0;
        double time_double = 0;
        double time_long_double = 0;

        current_utc_time(&tsi);
        for (g = 0; g<N; g++) for (i = 0; i < SIZE; i+=step) array_char[i] *= 3;    
        current_utc_time(&tsf);
        time_char = get_time_diff(tsi, tsf);

        //  =========== MODIFICA DESDE AQUI ============

        //  =========== MODIFICA HASTA AQUI ============

        fprintf(f,"%u %lf %lf %lf %lf %lf\n", step, time_char, time_int, time_long, time_double, time_long_double);
    }
    fclose(f);
    return 0; 

}

Overwriting files/cache_line_example.c


### Compila tu código

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
!make -f m/Makefile-p1.3 clean all

### Ejecuta tu código

**$\rightarrow$ ejecuta** (`shift` + `return`)
<br/>y espera a que acabe la ejecución, observa el tamaño de cada variable

In [ ]:
!./files/cache_line_example

### Carga los datos generados por el programa en C en un `DataFrame` de Python

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
file_name="files/cache_line.data"
df = pd.read_csv(file_name, sep=" ")
step_size           = df['step_size']
elapsed_time_int    = df['elapsed_time_int']
elapsed_time_long   = df['elapsed_time_long']
elapsed_time_char   = df['elapsed_time_char']
elapsed_time_double = df['elapsed_time_double']
elapsed_time_long_double = df['elapsed_time_long_double']
df

### Pinta los datos
observa como representamos los datos en **ESCALA LOGARÍTMICA** y pintamos el resultado ideal, en el que cada step size tarda la mitad del anterior

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
ideal_time = 1./np.power(2, np.array(range(len(step_size))))

ax = plt.figure(figsize=(10,5)).add_subplot(111)
pideal,  = ax.plot(df.index, np.log2(ideal_time), linewidth=3, alpha=0.5)
pchar,   = ax.plot(df.index, np.log2(elapsed_time_char), linewidth=3, alpha=0.5)
pint,    = ax.plot(df.index, np.log2(elapsed_time_int), linewidth=3, alpha=0.5)
plong,   = ax.plot(df.index, np.log2(elapsed_time_long), linewidth=3, alpha=0.5)
pdouble, = ax.plot(df.index, np.log2(elapsed_time_double), linewidth=3, alpha=0.5)
pldouble, = ax.plot(df.index, np.log2(elapsed_time_long_double), linewidth=3, alpha=0.5)
ax.set_xticks(df.index)
ax.set_xticklabels(step_size)
ax.set_xlabel("step size")
ax.set_ylabel("log secs")
ax.legend([pideal, pchar, pint, plong, pdouble, pldouble], 
          ["pendiente ideal", "char (1)", "int (4)", "long (8)", "double (8)", "long double (16)"])

# PREGUNTAS:
* ¿Con qué `step_size` se pierde la analogía con la pendiente ideal para cada tipo de dato?
* ¿Por qué con cada tipo de dato el comportamiento es distinto? ¿Qué relación hay?

--- TU RESPUESTA AQUí (double click) ---